<a href="https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/Pyserini%2BSciBERT_on_COVID_19_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyserini Demo on COVID-19 Dataset (Title + Abstract Index) 
# with HuggingFace Transformers based visualization


This notebook provides a demo on how to get started in searching the [COVID-19 Open Research Dataset](https://pages.semanticscholar.org/coronavirus-research) (release of 2020/03/20) from AI2.
In this notebook, we'll be working with the title + abstract index. 
Specifically, we're not indexing the full text (that'll come later, soon!).


In [1]:
from IPython.core.display import display, HTML

First, install Python dependencies

In [2]:
%%capture
!pip install pyserini==0.8.1.0
!pip install transformers

import json
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-13.0.2.jdk/Contents/Home"

Let's grab the pre-built index:

In [3]:
%%capture
!wget https://www.dropbox.com/s/j1epbu4ufunbbzv/lucene-index-covid-2020-03-27.tar.gz
!tar xvfz lucene-index-covid-2020-03-27.tar.gz

Sanity check of index size (should be 1.3G):

In [4]:
!du -h lucene-index-covid-2020-03-27

1.4G	lucene-index-covid-2020-03-27


Let's load BioBERT (https://arxiv.org/abs/1901.08746) from HuggingFace Transformers

In [5]:
import torch
import numpy
from tqdm import tqdm
from transformers import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
tokenizer = AutoTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed', do_lower_case=False)
model = AutoModel.from_pretrained('monologg/biobert_v1.1_pubmed')

You can use `pysearch` to search over an index. Here's the basic usage:

In [7]:
COVID_INDEX = 'lucene-index-covid-2020-03-27/'

def show_query(query):
    """HTML print format for the searched query"""
    return HTML('<br/><div style="font-family: Times New Roman; font-size: 20px;'
                'padding-bottom:12px"><b>Query</b>: '+query+'</div>')

def show_document(idx, doc):
    """HTML print format for document fields"""
    have_body_text = 'body_text' in json.loads(doc.raw)
    body_text = ' Full text available.' if have_body_text else ''
    return HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px">' + 
               f'<b>Document {idx}:</b> {doc.docid} ({doc.score:1.2f}) -- ' +
               f'{doc.lucene_document.get("authors")} et al. ' +
             # f'{doc.lucene_document.get("journal")}. ' +
             # f'{doc.lucene_document.get("publish_time")}. ' +
               f'{doc.lucene_document.get("title")}. ' +
               f'<a href="https://doi.org/{doc.lucene_document.get("doi")}">{doc.lucene_document.get("doi")}</a>.'
               + f'{body_text}</div>')

def show_query_results(query, searcher, top_k=10):
    """HTML print format for the searched query"""
    hits = searcher.search(query)
    display(show_query(query))
    for i, hit in enumerate(hits[:top_k]):
        display(show_document(i+1, hit))
    return hits[:top_k]   

In [8]:
from pyserini.search import pysearch

searcher = pysearch.SimpleSearcher(COVID_INDEX)
query = ('these differences reside in the molecular structure of spike proteins and some other factors. Which receptor combination(s) will cause maximum harm')
hits = show_query_results(query, searcher, top_k=10)

From the hits array, use `.lucene_document` to access the underlying indexed Lucene `Document`, and from there, call `.get(field)` to fetch specific fields, like "title", "doc", etc.
The complete list of available fields is [here](https://github.com/castorini/anserini/blob/master/src/main/java/io/anserini/index/generator/CovidGenerator.java#L46).

Let's extract contextualized vectors of queries and abstracts from SciBERT for highlighting relevant paragraphs.

First, extract the contextualized vectors of the query above:

$$q_1, \ldots, q_T = \text{SciBERT}(\text{query})$$

In [9]:
def extract_scibert(text, tokenizer, model):
    text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    text_words = tokenizer.convert_ids_to_tokens(text_ids[0])[1:-1]

    n_chunks = int(numpy.ceil(float(text_ids.size(1))/512))
    states = []
    
    for ci in range(n_chunks):
        text_ids_ = text_ids[0, 1+ci*512:1+(ci+1)*512]            
        torch.cat([text_ids[0, 0].unsqueeze(0), text_ids_])
        if text_ids[0, -1] != text_ids[0, -1]:
            torch.cat([text_ids, text_ids[0,-1].unsqueeze(0)])
        
        with torch.no_grad():
            state = model(text_ids_.unsqueeze(0))[0]
            state = state[:, 1:-1, :]
        states.append(state)

    state = torch.cat(states, axis=1)
    return text_ids, text_words, state[0]

In [10]:
query_ids, query_words, query_state = extract_scibert(query, tokenizer, model)

Second, let's extract contextualized vectors of all the paragraphs from the hit #7:

$$p_1^k, \ldots, p_{T_k}^k = \text{SciBERT}(\text{paragraph}^k)$$

In [11]:
ii = 0
doc_json = json.loads(hits[ii].raw)

paragraph_states = []
for par in tqdm(doc_json['body_text']):
    state = extract_scibert(par['text'], tokenizer, model)
    paragraph_states.append(state)

100%|██████████| 27/27 [00:12<00:00,  2.14it/s]


We then compute the cosine similarity matrix between the query and each paragraph:

$$A^k = [a^k_{ij}] \in \mathbb{R}^{|\text{query}| \times |\text{paragraph}^k|},$$

where

$$a^k_{ij} = \frac{q_i^\top p_j^k}{\| q_i \| \| p_j^k \|}$$


In [12]:
def cross_match(state1, state2):
    state1 = state1 / torch.sqrt((state1 ** 2).sum(1, keepdims=True))
    state2 = state2 / torch.sqrt((state2 ** 2).sum(1, keepdims=True))
    sim = (state1.unsqueeze(1) * state2.unsqueeze(0)).sum(-1)
    return sim

In [13]:
sim_matrices = []
for pid, par in tqdm(enumerate(doc_json['body_text'])):
    sim_score = cross_match(query_state, paragraph_states[pid][-1])
    sim_matrices.append(sim_score)

27it [00:00, 83.24it/s]


Let's retrieve the most relevant paragraphs first, where define the top-$M$ most relevant paragraphs as 

$$\arg\text{top-$M$}_{k=1}^K \max_{i=1,\ldots,|\text{query}|} \max_{j=1,\ldots, |\text{paragraph}^k|} A_{ij}^k$$

that is, a paragraph with the highly matched words to the query words is considered relevant.

In [14]:
paragraph_relevance = [torch.max(sim).item() for sim in sim_matrices]

# Select the index of top 5 paragraphs with highest relevance
rel_index = numpy.argsort(paragraph_relevance)[-5:][::-1]

In [15]:
def show_sections(section, text):
    """HTML print format for document subsections"""
    return HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px; margin-left: 15px">' + 
        f'<b>{section}</b> -- {text.replace(" ##","")} </div>')

display(show_query(query))
display(show_document(ii, hits[ii]))
for ri in numpy.sort(rel_index):
    display(show_sections(doc_json["body_text"][ri]['section'], " ".join(paragraph_states[ri][1])))

We want to look at more details by highlighting relevant phrases in each paragraph, where we define relevant phrases for each paragraph as

$$\arg\text{top-$M$}_{j=1,\ldots, |\text{paragraph}^k|} \max_{i=1,\ldots,|\text{query}|} A_{ij}^k$$

that is, any word that had a high similarity to each of the query words is considered relevant. given these words, we highlight a window of 10 surrounding each of them.

In [16]:
def highlight_paragraph(ptext, rel_words, max_win=10):
    para = ""
    prev_idx = 0
    for jj in rel_words:
        
        if prev_idx > jj:
            continue
        
        found_start = False
        for kk in range(jj, prev_idx-1, -1):
            if ptext[kk] == "." and (ptext[kk+1][0].isupper() or ptext[kk+1][0] == '['):
                sent_start = kk
                found_start = True
                break
        if not found_start:
            sent_start = prev_idx-1
            
        found_end = False
        for kk in range(jj, len(ptext)-1):
            if ptext[kk] == "." and (ptext[kk+1][0].isupper() or ptext[kk+1][0] == '['):
                sent_end = kk
                found_end = True
                break
                
        if not found_end:
            if kk >= len(ptext) - 2:
                sent_end = len(ptext)
            else:
                sent_end = jj
        
        para = para + " "
        para = para + " ".join(ptext[prev_idx:sent_start+1])
        para = para + " <font color='blue'>"
        para = para + " ".join(ptext[sent_start+1:sent_end])
        para = para + "</font> "
        prev_idx = sent_end
        
    if prev_idx < len(ptext):
        para = para + " ".join(ptext[prev_idx:])

    return para

In [17]:
display(show_query(query))

display(show_document(ii, hits[ii]))

for ri in numpy.sort(rel_index):
    sim = sim_matrices[ri].data.numpy()
    
    # Select the two highest scoring words in the paragraph
    rel_words = numpy.sort(numpy.argsort(sim.max(0))[-2:][::-1])
    p_tokens = paragraph_states[ri][1]
    para = highlight_paragraph(p_tokens, rel_words)
    display(show_sections(doc_json["body_text"][ri]['section'], para))